# Exercício 1: Ações e Commodities
###  Implementando Uma Estratégia Replicante 
<sub>Uirá Caiado. 11 de Maio, 2016<sub>

#### Resumo

_Neste projeto vou implementar uma estratégia de hedging dinâmico auto-financiável que replique o payoff de diferentes derivativos. Devido à natureza aleatória do ativo objeto dos contratos testados, este será modelado como um processo estocástico. Uma vez implementado o modelo, será realizada um série de simulações de Monte Carlo para calcular o preço jutso dos derivativos desejados._

## 1. Introdução

Devido à incerteza inerente ao mercado financeiro, é necessário levar em conta esta flutuação aleatória nos modelos criados para descrever diferentes instrumentos. Para tanto, frequentemente descreve-se o comportamento de um ativo como um processo estocático. \cite{Wilmont} oferece a seguinte intuição sobre a forma geral de Equações Diferenciais Estocásticas (EDE):

$$\mathrm{d}\, Alguma\,Coisa = Deterministico\,\mathrm{d}t + Aleatório\,\mathrm{d}X $$


Onde $Alguma\,Coisa$ é o que se quer modelar (como preço de uma ação, juros, ...), $Deterministico$ uma função que descreve como  $Alguma\,Coisa$ muda quando se retira a incerteza e $Aleatório$ outra função que descreve quão incerto $Alguma\,Coisa$ coisa é e $X$ é uma variável aleatória com distribuição Normal Padrão.

Desta forma, considerando um ativo-objeto cujo a dinâmica do preço é descrita pela seguinte EDE:

$$\frac{\mathrm{d} S_t}{S_t}=\mu\cdot \mathrm{d}t + \sigma\cdot \mathrm{d}W_{t}$$

Será vendido um derivativo com característica européia cujo payoff é descrito por uma função qualquer $V_T=V(T, S_T)$, onde $T$ é o vencimento do derivativo. $S_T$ é o preço do ativo-objeto em $T$ e é possível negociar qualquer quantidade dele em qualquer instante. Não há custo de transação (corretagem, emolumento, bid-ask spread, etc) e posições vendidas a descoberto no subjacente são permitidas, não havendo custos associados.

Pede-se que se implemente uma estratégia dinâmica de hedging auto-financiável que replique o payoff do derivativo vendido. A estratégia replicante será rebalanceada $N$ vezes (incluíndo a carteira inicial).

A simulação deve ser feita para os payoffs abaixo. $K$ é o Strike da opção.
- $V(T, S_T)=ln(S_T)$
- $V(T, S_T)=(ln(S_T))^2$
- $V(T, S_T)=(S_T-K)^2$
- $V(T, S_T)=\mathbf{1}_{S_T > K}$
- $V(T, S_T)=max(S_T-K, 0)$


## 2. Implementação

Nesta sessão vou obterei as equações necessárias para a realização deste projeto e implementarei os códigos necessários para realização das simulações.

### 2.1. Descrevendo o Preço das Opções

...

### 2.2. Discretizando o Processo Estocástico

...

In [2]:
# importa bibliotecas necessarias
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
# configura graficos
%matplotlib inline




## 3. A Simulação


## 4. Conclusão

## 5. Reflexões

_Style notebook and change matplotlib defaults_

In [1]:
#loading style sheet
from IPython.core.display import HTML
HTML( open('ipython_style.css').read())

In [2]:
#changing matplotlib defaults
%matplotlib inline
import seaborn as sns
sns.set_palette("deep", desat=.6)
sns.set_context(rc={"figure.figsize": (8, 4)})
sns.set_style("whitegrid")
sns.set_palette(sns.color_palette("Set1", 10))